In [60]:
import pandas as pnd
import plotly.express as px
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

<ipython-input-60-7ace8cc495f5>:7: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

<ipython-input-60-7ace8cc495f5>:8: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In [3]:
data=pnd.read_csv('uber vs lyft.csv')
pnd.set_option('max_columns', None)
data=data.iloc[:,1:]

In [4]:
uber_data=data[data['cab_type']=='Uber']
uber_data=uber_data.reset_index()
uber_data=uber_data.iloc[:,1:]

In [5]:
lyft_data=data[data['cab_type']=='Lyft']
lyft_data=lyft_data.reset_index()
lyft_data=lyft_data.iloc[:,1:]

In [6]:
data

,id,timestamp,hour,day,month,datetime,source,destination,cab_type,product_id,name,price,distance,surge_multiplier,temperature,long_summary,icon,visibility,temperatureHigh,temperatureLow,apparentTemperatureHigh,apparentTemperatureLow,latitude,longitude,days name,time of drive
0,a7b50600-c6c5-4e6c-bea9-4487344196d4,1.543204e+09,3,26,11,2018-11-26 03:40:46,North Station,Haymarket Square,Uber,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,7.000000,0.56,1.0,41.83,Rain in the morning and afternoon.,fog,1.685,46.15,40.15,43.17,39.21,42.3429,-71.1003,Monday,Late Night
1,9962f244-8fce-4ae9-a583-139d5d7522e1,1.543204e+09,3,26,11,2018-11-26 03:40:46,Theatre District,North End,Uber,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi,16.545125,1.57,1.0,41.83,Rain in the morning and afternoon.,fog,1.685,46.15,40.15,43.17,39.21,42.3429,-71.1003,Monday,Late Night
2,4aa68a5d-abc0-4fdf-a47f-0003617afbae,1.543204e+09,3,26,11,2018-11-26 03:40:46,North End,West End,Lyft,lyft,Lyft,7.000000,1.23,1.0,41.83,Rain in the morning and afternoon.,fog,1.685,46.15,40.15,43.17,39.21,42.3429,-71.1003,Monday,Late Night
3,ef8b695c-c24d-4ac1-b3fe-4aa1a7ed79f4,1.543204e+09,3,26,11,2018-11-26 03:40:46,Boston University,Beacon Hill,Lyft,lyft_premier,Lux,19.500000,2.66,1.0,41.83,Rain in the morning and afternoon.,fog,1.685,46.15,40.15,43.17,39.21,42.3429,-71.1003,Monday,Late Night
4,89f35ef7-7129-483d-b3e6-d89afdf6946d,1.543204e+09,3,26,11,2018-11-26 03:40:46,North End,West End,Lyft,lyft_line,Shared,5.000000,1.23,1.0,41.83,Rain in the morning and afternoon.,fog,1.685,46.15,40.15,43.17,39.21,42.3429,-71.1003,Monday,Late Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693066,e299c3bf-a429-4b19-af4a-ebd8e9ad74f7,1.545161e+09,19,18,12,2018-12-18 19:15:10,Financial District,Haymarket Square,Uber,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,7.000000,1.25,1.0,32.34,Mostly cloudy throughout the day.,clear-day,9.883,32.97,20.95,22.90,14.26,42.3519,-71.0551,Tuesday,Evening
693067,20caa061-2ded-49f8-882b-1e7eae6285ff,1.545161e+09,19,18,12,2018-12-18 19:15:10,Fenway,Theatre District,Uber,6d318bcc-22a3-4af6-bddd-b409bfce1546,Black SUV,33.500000,2.66,1.0,32.34,Mostly cloudy throughout the day.,clear-day,9.883,32.97,20.95,22.90,14.26,42.3519,-71.0551,Tuesday,Evening
693068,7f1cbf41-2136-4e37-889d-dd0dfff02d38,1.545161e+09,19,18,12,2018-12-18 19:15:10,Haymarket Square,Back Bay,Uber,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,11.500000,2.30,1.0,32.34,Mostly cloudy throughout the day.,clear-day,9.883,32.97,20.95,22.90,14.26,42.3519,-71.0551,Tuesday,Evening
693069,8c28dc35-c4a8-41e8-abe8-d5d65849448d,1.545161e+09,19,18,12,2018-12-18 19:15:10,Northeastern University,Beacon Hill,Lyft,lyft_luxsuv,Lux Black XL,34.000000,3.00,1.0,32.34,Mostly cloudy throughout the day.,clear-day,9.883,32.97,20.95,22.90,14.26,42.3519,-71.0551,Tuesday,Evening


In [92]:
uber_days=uber_data['days name'].value_counts()
lyft_days=lyft_data['days name'].value_counts()
d=uber_data.drop_duplicates(subset = ['distance'])
d=d.sort_values('distance', ascending=True)
d['name'].value_counts()

Black        65
Taxi         64
Black SUV    62
UberPool     62
UberX        61
UberXL       54
WAV          54
Name: name, dtype: int64

In [97]:
all_continents = d.name.unique()

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Checklist(
        id="checklist",
        options=[{"label": x, "value": x} 
                 for x in all_continents],
        value=all_continents[:],
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id="line-chart"),
])

@app.callback(
    Output("line-chart", "figure"), 
    [Input("checklist", "value")])
def update_line_chart(continents):
    mask = d.name.isin(continents)
    fig = px.line(d[mask], x="distance", y="price", color='name')
    return fig
app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 48] Address already in use

In [95]:
d = d.query("name in ['UberX', 'Black','WAV','Taxi','Black SUV','UberPool','UberXL']")
fig=px.line(d,x='distance',y='price',color="name")
fig.show()